SyntaxError: invalid syntax (Temp/ipykernel_26996/1469695481.py, line 1)

In [2]:

import gym
from gym import logger as gymlogger
from gym.wrappers import RecordVideo
gymlogger.set_level(40) #error only
#import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

In [3]:
env = gym.make("CartPole-v1")

## Initialize variables

In [95]:
LEARNING_RATE = 0.1

DISCOUNT = 0.95
EPISODES = 60000
total = 0
total_reward = 0


Observation = [30, 30, 50, 50]
np_array_win_size = np.array([0.25, 0.25, 0.01, 0.1])

epsilon = 0.9
epsilon_decay = 0.99
epsilon_min = 0.5
epsilon_max = 0.9

prev_mean = 0
obs = env.reset()


new_env = env.step(0)[0]
print(new_env)
print(round(new_env[2], 2))

[ 8.0382861e-03 -2.4367590e-01  1.5835793e-04  2.6480407e-01]
0.0


## Create Q table

In [96]:
q_table = np.random.uniform(low=0, high=1, size=(Observation + [env.action_space.n]))
q_table.shape

(30, 30, 50, 50, 2)

## Getting discrete state

In [97]:
def get_discrete_state(state):
    
    #Given a state, round to the nearest 0.25
    #divide by the window size to find which bucket it belongs to
    #discrete_state =np.array([np.round(state[0]*4)/4, np.round(state[1] *4) /4, np.round(state[2], 1) ,np.round(state[3], 1)])
    #discrete_state =np.array([np.round(state[0]*2)/2, np.round(state[1] *2) /2, np.round(state[2]* 2)/2 ,np.round(state[3]* 2)/2])
    discrete_state = state/np_array_win_size+ np.array([15,10,1,10])
    #discrete_state = (discrete_state/np_array_win_size)
    #print(discrete_state)
    return tuple(discrete_state.astype(int))


print(get_discrete_state([4, -2, 0.418, 0.63751878]))

(31, 2, 42, 16)


## Running

In [98]:
for episode in range(EPISODES): 

    discrete_state = get_discrete_state(env.reset()[0])  
  
    done = False
    episode_reward = 0 
    
    #update every 2000 episodes
    if episode % 2000 == 0: 
        print("Episode: " + str(episode))
    

    while not done: 
        #Always get max for now
        if np.random.random() < epsilon:
            action = np.argmax(q_table[discrete_state]) 
        else:
            action = np.random.randint(0, env.action_space.n) 
        new_env = env.step(action)
        new_state = new_env[0]
        reward = new_env[1]
        done = new_env[2]
        episode_reward += reward 

        new_discrete_state = get_discrete_state(new_state)

        if not done: 
            
            max_future_q = np.max(q_table[new_discrete_state])

            current_q = q_table[discrete_state + (action,)]

            new_q = (1-LEARNING_RATE)*current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)
            #new_q = current_q + LEARNING_RATE * (reward + DISCOUNT*(max_future_q) - current_q)
            q_table[discrete_state + (action,)] = new_q

        discrete_state = new_discrete_state

    total_reward += episode_reward

    
    if episode % 1000 == 0: 
        
        mean_reward = total_reward / 1000    
        print("Mean Reward: " + str(mean_reward))
        if (mean_reward < (prev_mean * 0.95) and epsilon > epsilon_min):
            print("Decaying")
            epsilon *= epsilon_decay
        if (mean_reward > (prev_mean * 1.1) and epsilon < epsilon_max):
            print("improving")
            epsilon /= epsilon_decay
        prev_mean = mean_reward
        total_reward = 0
        

env.close()

Episode: 0
Mean Reward: 0.022
Mean Reward: 28.029
Episode: 2000
Mean Reward: 28.524
Mean Reward: 29.146
Episode: 4000
Mean Reward: 30.382
Mean Reward: 31.847
Episode: 6000
Mean Reward: 38.205
Mean Reward: 39.995
Episode: 8000
Mean Reward: 48.926
Mean Reward: 53.287
Episode: 10000
Mean Reward: 64.394
Mean Reward: 80.574
Episode: 12000
Mean Reward: 92.056
Mean Reward: 120.26
Episode: 14000
Mean Reward: 116.311
Mean Reward: 157.515
Episode: 16000
Mean Reward: 153.507
Mean Reward: 165.058
Episode: 18000
Mean Reward: 138.163
Decaying
Mean Reward: 143.677
Episode: 20000
Mean Reward: 176.849
improving
Mean Reward: 218.282
Episode: 22000
Mean Reward: 231.947
Mean Reward: 181.507
Decaying
Episode: 24000
Mean Reward: 214.97
improving
Mean Reward: 231.295
Episode: 26000
Mean Reward: 256.854
Mean Reward: 227.128
Decaying
Episode: 28000
Mean Reward: 212.81
Decaying
Mean Reward: 215.021
Episode: 30000
Mean Reward: 187.982
Decaying
Mean Reward: 190.334
Episode: 32000
Mean Reward: 191.657
Mean Reward:

KeyboardInterrupt: 